# The causal Impact Of Capital Punishment Upon Murders
## Is anybody scared of the death penalty?

### EC499
### Isaac Updike & Ryan Granet



## Introduction

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Of the most controversial issues in criminal justice is capital punishment. The US distinguishes itself among developed democratic nations by sentencing people to death, being surpassed in executions only by China, Iran, Saudi Arabia, Iraq, and Egypt. The justice system administers punishments to convicted criminals with four goals in mind: retribution, incapacitation, deterrence, and rehabilitation. Capital punishment arguably accomplishes incapacitation and retribution, and abandons rehabilitation albeit at a high cost; a human life. Yet the alternative to death sentencing, life sentencing, similarly accomplishes these goals whilst preserving the possibility of rehabilitation.  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; In considering the viability and ethical implications of capital punishment within the US penal system, one must examine the degree to which it accomplishes the fourth goal, deterrence. Over the last decade, global executions have trended lower. Some believe that the threat of execution for committing certain crimes has been a deterring force. Though, in the United States, we have seen some states abandon capital punishment, even going as far as to rule it unconstitutional. The goal of this paper is to examine the effects of capital punishment in the United States, in an attempt to explain the effectiveness as a policy for deterring murders. According to Levitt (2004), the crime rates throughout the 90s fell sharply due to factors unrelated to capital punishment. Levitt continues to describe that criminals engage in activities that run higher chances of death than being convicted of murder. The higher every-day risks involved with being a criminal means that rational criminals would not be deterred by such low risk of being convicted and executed for their crimes. This paper will use county-level panel data to track murders and executions across 16 years. 


## Methods: Econometrix

The models developed rely on a county level panel dataset consisting of observations from years 1980 through 1996. The dependent variable is murdrate, the number of murders per 10,000 people in a county. The explanatory variable of interest is execrate, the number of executions per 10,000 people in a county. In order to build the model, it was important to understand which factors were going to be left unaccounted for. The relationship between police and the community, the effectiveness of policing and investigating, and the willingness to give information to the police without fear of retaliation are factors that are unmeasured in our data. However, it is assumed that those factors are all important in the success of police arresting suspects and can be grouped into a new variable arrestperc   

The Fixed Effects estimator is a favorable measure of causal effects when working with panel datasets with many time periods. Using such an estimator avoids omitted variable bias by controlling for between group differences using the ‘within’ transformation. This takes the across period average for each


$$ log\text{_}murdrate_{it} = \beta_{0}  + \beta_{1} execrate_{it} + \beta_{2} execrate_{i,t-1} + \beta_{3} execrate_{i,t-2}+  \beta_{4} log\text{_}arrestper_{it} + \beta_{5} arrests_{it} + \beta_{6} arrests2_{it} + \beta_{7} density_{it} + \beta_{8} popul_{it} + \beta_{9} perc1019_{it} + \beta_{10} perc2029_{it} + \beta_{11} percblack_{it} + \beta_{12} percmale_{it} + \beta_{13} percmale*percblack{it} \beta_{14} rpcpersinc_{it} + \beta_{15} rpcunemins_{it} + \beta_{16} rpcincmaint_{it} +  \beta_{15} year{it} + \alpha_{it} + u_{it}$$ 

### Variable descriptions
$ log\text{_}murdrate_{it} $ is the log of murders per 10,000. which is crucial in understanding whether capitol punishment has a detterent effect upon crime since murder is the predominant crime which people are sentenced to death for. 
$ \alpha_{it} $ represents between-group differences in counties, unobserved variables that may contribute to $ log\text{_}murdrate_{it}$ but don't change over time such as culture.
$ u_{it} $ is idiosyncratic error specififc to i,t such as availability of firearms, availability of police officers, or mental health services. Sentencing is way before this all too

**Control Variables**
 
To limit bias in the model, we control for other factors that may influence the decision to murder. $arrests$ indicates the number of arrests involving homicide charges brought against a murder suspect, which could act as a detterent effecct on murder. $ arrestper_{it} $ is the number of arrests as a fraction of the total murders, added to control for the relative efficiency of policing and law enforcment engagement a similar detterent effect. $arrests2_{it}$ is the squared $arrests_{it}$ allows for diminishing effects of arrests. To control for demographic differences across counties we included $density_{it}$ for , persons per square mile,$popul_{it}$ as total population, $perc1019_{it}$ & $perc2029_{it}$ are perecent age 10-19 & 20-29 respectively , $percblack_{it}$ is percent population black, $percmale_{it} $ percent male, and $percmale*percblack{it}$ is an interaction term between the percent black & male variables. $rpcpersinc_{it}$ reports real per capita (RPC) personal income, $ rpcunemins_{it} $ is RPC unemployment insurance, and $rpcincmaint_{it}$ RPC income maintenence, which all serve as proxis for the general welfare, poverty, and social safteynet of a county.  

The measure of capital punishment is the execution rate, reported as $execrate_{it}$. In order to account for a lagging effect of excutions, wherein the news of a convict being executed takes time before it affects the murder rate, the model incorperates the the previous 3 years of execution data at each estimated $\hat{y}$ through $exec. L1$ where the estimated effect of $exec. L1$ at time $t$ is the effect of $exec_{i,t-1}$ on $\hat{y}$.



## Data description

The data in this model comes primarily from 3 sources: DeathPenaltyinfo.org, the Uniform Crime Reports (UCR), U.S. Census Bureau. DeathPenaltyinfo.org is a think-tank centered in Washington DC, focusing on issues concerning capital punishment, they publish data on all executions performed in the United States going back to 1977 by county. The Uniform Crime Reports is a program operated by the FBI to publish administrative data and analysis on law enforcement activity. The U.S. Census Bureau collects population count estimates by city and county every year, and as part of the Current Population Survey collects data about personal income, employment and government benifits. each of these three datasets are indexed by state and county fips and combined in long format such that each row is a list of $y_{it}$ and $X_{it}$.  

While the data include most US counties and for each of the 11 years, there were still only a total of 256 executions. Most death sentences take over a decade to be carried out, and very few counties ever see any executions. Murder was much more prevalent, with 272159 total murders



In [1]:
import ipystata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

IPyStata is loaded in batch mode.


Load in execution data from [Death Penalty Information Center Exectuion Database](https://docs.google.com/spreadsheets/d/e/2PACX-1vQDMoiJXVldahXzL4S037MGb7DgZMfeqfrR-zYtDJ_U-Sd6FS35W7WU_6N8pXBOm9NOBGIM8BEui37x/pub?gid=0&single=true&output=csv) to attempt to expand dataset


## Grab data

In [38]:
df = pd.read_stata('data/countymurders.dta')
df.head(4)

,arrests,countyid,density,popul,perc1019,perc2029,percblack,percmale,rpcincmaint,rpcpersinc,rpcunemins,year,murders,murdrate,arrestrate,statefips,countyfips,execs,lpopul,execrate
0,2.0,1001,54.049999,32216.0,20.629999,15.28,22.330000,40.250000,167.669998,8780.799805,29.160000,1980,2,0.620810,0.620809,1,1,0,10.380219,0.0
1,3.0,1001,53.660000,31984.0,20.190001,15.55,22.070000,40.360001,167.990005,8232.799805,43.919998,1981,1,0.312656,0.937969,1,1,0,10.372991,0.0
2,2.0,1001,53.750000,32036.0,19.660000,15.73,21.799999,40.419998,166.630005,8327.610352,71.410004,1982,3,0.936446,0.624298,1,1,0,10.374616,0.0
3,7.0,1001,53.779999,32056.0,19.100000,15.88,21.530001,40.470001,176.529999,8545.549805,72.220001,1983,7,2.183679,2.183679,1,1,0,10.375239,0.0


Exclude counties without any murders at all

In [40]:
zeros = [] # for data quality testing 

for i in df['countyid'].unique():
    
    cn = df[df['countyid']==i]
    
    if sum(cn['murders'])==0:
        zeros.append(i)
        df.drop(df[df['countyid']==i].index,inplace=True)


Send the dataframe to stata with the -d command 

Declare the it as panel data to stata with panel/time varibles

Use xtreg, fe to define a linear model using the fixed-effects estimator

In [58]:
%%stata -d df

# generate new variable
gen arrestperc = (arrests / murders)

gen execrate2 = execrate * execrate

gen log_murdrate = log(murdrate)

gen log_exrate = log(execrate)

gen arrests2 = arrests * arrests

gen log_arrestper = log(arrestperc)

xtset countyid year
xtreg log_murdrate execrate L.execrate L2.execrate c.percmale#c.percblack log_arrestper arrests arrests2 density lpopul perc1019 perc2029 percblack percmale rpcpersinc rpcunemins rpcincmaint i.year, fe vce(cluster countyid)


Unknown #command(14,814 missing values generated)
(14,546 missing values generated)
(36,039 missing values generated)
(465 missing values generated)
(17,765 missing values generated)
       panel variable:  countyid (strongly balanced)
        time variable:  year, 1980 to 1996
                delta:  1 unit
>  fe vce(cluster countyid)

Fixed-effects (within) regression               Number of obs     =     16,148
Group variable: countyid                        Number of groups  =      2,006

R-sq:                                           Obs per group:
     within  = 0.2600                                         min =          1
     between = 0.5271                                         avg =        8.0
     overall = 0.3403                                         max =         15

                                                F(30,2005)        =     114.03
corr(u_i, Xb)  = -0.7592                        Prob > F          =     0.0000

                                     (Std

## Reuslts

After using a fixed effects regression model with a cluster robust standard error, we were able to estimate the effects on the murder rate and eliminate the errors a and u. The main variable in question is the execution rate, resembling capital punishment and it’s lasting effects on murder rates. The model estimates that B1 = -0.0896, meaning that an increase in the murder rate by 1/10000 would decrease the murder rate by  8.9%, all else equal. The magnitude on B1 is quite large, seeing that the county with the highest execrate being 2.39, giving a 20.55% decrease in the murder rate. However, when considering it’s statistical significance, the model estimates a p-value of 0.603 for B1, rendering it not statistically significant in lowering the murder rate. The following lag variables, B2 and B3, coefficients are -0.067 and -0.184 respectively. This shows a larger impact in the third year after an execution, yet remains still statistically significant in deterring murder with a p-value of 0.224. Based on the model, executions, in past or present, have not had an impact in reducing the murder rate. This finding is not all too surprising, since there have only been 256 executions, which is small compared to the number of murders. The coefficient on log_arrestper, B5, is -0.429, meaning that an increase in 1% of arrestper would lead to a decrease in the murder rate by 0.429 percent. The magnitude is important because the high end of arrestper reaches 50. This means that counties with the highest arrests made per murder committed would see a decrease in the murder rate by 21.4% and has a p-value of 0.000, making it statistically significant in reducing the murder rate. It is interesting to see a positive coefficient on lpopul, B9, meaning that an increase in the population will decrease the murder rate at a faster rate than would effective policing. The p-value is 0.000, so it is statistically significant. Other demographic control variables are not statistically significant, below the 10% level. When considering the impacts of economic status on murder rates, the only variable that is statistically significant is rpcpersinc, B14. B14 is estimated to be 0.00002 with a p-value of 0.01, meaning it is statistically significant at the 1% level. This means that there is a 75% reduction in murders in a county that is at the top of the rpcpersinc versus the bottom. 


## Conclusion

After estimating coefficients on the model, the goal was to observe if the policy effects of capital punishment lowered the murder rate through increasing the costs of committing murders, if caught. The United States has a large amount of murders compared to executions, seeing that of the 272,159 murders there were only 256 executions. According to the estimates, capital punishment fails to be statistically significant in deterring murder. If there were a larger amount of executions, like in China where it is estimated to execute thousands per year, there may be a more significant impact in reducing the murder rates, whereas most counties in the US don’t experience a death sentence. A rational criminal still realizes the costs involved with getting caught for murder, facing extensive prison time. $arrestper$ captures implied risk of being caught, signifying the effectiveness of policing at arresting murderers. Since police seem to be better at arresting murderers than at executing them, $arrestper$  is more effective in raising the costs of committing homicides. There are considerable barriers to carrying out successful death sentences, such as juror ethical boundaries, appellate decisions, and long wait times which may diminish the effectiveness of it as a deterrent.
 
 This leads to the conclusion that capital punishment as a policy for deterring murder is statistically insignificant and that policies aimed at increasing the efficency of policing would be a more effective tool. The next step in finding causing effects on murder rates would be to obtain data on number of police officers and any other factors that may increase the effectiveness of police. Obtaining state-level panel data containing murders and other variables through the time before and after capital punishment was abandoned may lead to more answers through discontinuity analysis.
